
# 10 — Full RAG + Agents + MCP (Integrated Demo)

This notebook demonstrates a **small but complete integrated pipeline**:

- RAG backend over a tiny corpus
- Planner → Retriever → Answer agents
- MCP-style ToolRegistry (calculator + search)
- LangGraph workflow to orchestrate:
  - RAG path
  - Tool path
  - Final answer


## 0. Setup

In [ ]:

!pip install langchain langchain-openai langchain-community chromadb langgraph --quiet

import os
from typing import Dict, Any, TypedDict, Callable
from dataclasses import dataclass

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o-mini")
emb = OpenAIEmbeddings()


## 1. RAG Backend (Tiny Corpus)

In [ ]:

os.makedirs("full_rag_agents_mcp_data", exist_ok=True)

with open("full_rag_agents_mcp_data/corpus.txt", "w") as f:
    f.write("RAG retrieves external documents and feeds them into the LLM for grounded answers.\n")
    f.write("Agents can plan, decide which tools to call, and manage multi-step reasoning.\n")
    f.write("MCP provides a protocol for exposing tools and calling them from agents.\n")

loader = TextLoader("full_rag_agents_mcp_data/corpus.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = splitter.split_documents(docs)

db = Chroma.from_documents(chunks, emb)
retriever = db.as_retriever(search_kwargs={"k": 3})


## 2. RAG Agents (Planner → Retriever → Answer)

In [ ]:

def planner_agent(question: str) -> str:
    prompt = f"""You are a Planner Agent.

Write a short plan (1–2 sentences) describing how to answer this question
using the RAG knowledge base.

Question: {question}
"""
    return llm.invoke(prompt).content

def retriever_agent(plan: str) -> str:
    docs = retriever.get_relevant_documents(plan)
    return "\n".join(d.page_content for d in docs)

def answer_agent(question: str, context: str) -> str:
    prompt = f"""You are an Answer Agent.

Use ONLY the context to answer the question concisely.

Context:
{context}

Question: {question}

Answer:
"""
    return llm.invoke(prompt).content

# Quick test
print(planner_agent("Explain how RAG, agents, and MCP relate."))
ctx_test = retriever_agent("Retrieve RAG + agents + MCP relationship.")
print(answer_agent("Explain how RAG, agents, and MCP relate.", ctx_test))


## 3. MCP-Style ToolRegistry + Tools

In [ ]:

class ToolRegistry:
    def __init__(self):
        self.tools: Dict[str, Callable[..., Any]] = {}

    def register(self, name: str, fn: Callable[..., Any]):
        self.tools[name] = fn

    def call(self, name: str, **kwargs) -> Any:
        if name not in self.tools:
            raise ValueError(f"Tool '{name}' not found")
        return self.tools[name](**kwargs)

tool_registry = ToolRegistry()

# Example tools
def tool_calculator(expression: str) -> str:
    try:
        result = eval(expression, {"__builtins__": {}})
        return f"Result of {expression} = {result}"
    except Exception as e:
        return f"Error: {e}"

def tool_fake_search(query: str) -> str:
    return f"[Fake search results for '{query}': RAG docs, Agents docs, MCP docs]"

tool_registry.register("calculator", tool_calculator)
tool_registry.register("search", tool_fake_search)

print(tool_registry.call("calculator", expression="2 + 3 * 10"))
print(tool_registry.call("search", query="agentic RAG"))


## 4. Tool-Using Agent

In [ ]:

@dataclass
class ToolDecision:
    tool_name: str
    args: Dict[str, Any]

class ToolAgent:
    def __init__(self, llm: ChatOpenAI, registry: ToolRegistry):
        self.llm = llm
        self.registry = registry

    def decide(self, question: str) -> ToolDecision | None:
        q = question.lower()
        if "calculate" in q or "calc" in q or any(c in q for c in ["+", "-", "*", "/"]):
            expr = question.replace("calculate", "").replace("calc", "").strip()
            return ToolDecision(tool_name="calculator", args={"expression": expr or question})
        if "search" in q or "lookup" in q or "google" in q:
            return ToolDecision(tool_name="search", args={"query": question})
        return None

    def run(self, question: str) -> str:
        decision = self.decide(question)
        if not decision:
            # fallback: just let the LLM answer
            return self.llm.invoke(question).content

        tool_result = self.registry.call(decision.tool_name, **decision.args)
        prompt = f"""You are an assistant that uses tools.

User question:
{question}

Tool called: {decision.tool_name}
Tool result:
{tool_result}

Explain the result clearly to the user.
"""
        return self.llm.invoke(prompt).content

tool_agent = ToolAgent(llm, tool_registry)
print(tool_agent.run("Calculate 10 * (5 + 2)"))
print(tool_agent.run("Search about MCP with agents"))


## 5. LangGraph Integrated Workflow

In [ ]:

class FullState(TypedDict):
    question: str
    mode: str
    plan: str
    context: str
    answer: str

def router_node(state: FullState) -> FullState:
    q = state["question"].lower()
    # Very simple routing heuristic: if math/search-like, use tools, else RAG
    if any(x in q for x in ["calculate", "calc", "+", "-", "*", "/", "search", "lookup", "google"]):
        state["mode"] = "TOOLS"
    else:
        state["mode"] = "RAG"
    return state

def rag_node(state: FullState) -> FullState:
    plan = planner_agent(state["question"])
    ctx = retriever_agent(plan)
    ans = answer_agent(state["question"], ctx)
    state["plan"] = plan
    state["context"] = ctx
    state["answer"] = ans
    return state

def tool_node(state: FullState) -> FullState:
    ans = tool_agent.run(state["question"])
    state["plan"] = ""
    state["context"] = ""
    state["answer"] = ans
    return state

graph = StateGraph(FullState)
graph.add_node("router", router_node)
graph.add_node("rag_flow", rag_node)
graph.add_node("tool_flow", tool_node)

graph.set_entry_point("router")

def branch_from_router(state: FullState):
    if state["mode"] == "TOOLS":
        return "tool_flow"
    return "rag_flow"

graph.add_conditional_edges(
    "router",
    branch_from_router,
    {
        "rag_flow": "rag_flow",
        "tool_flow": "tool_flow",
    },
)

graph.add_edge("rag_flow", END)
graph.add_edge("tool_flow", END)

workflow = graph.compile()

print(workflow.invoke({"question": "Explain how RAG, agents, and MCP relate."}))
print(workflow.invoke({"question": "Calculate 7 * (3 + 4)"}))



## End of Notebook 10 — Full RAG + Agents + MCP

You now have:

- A tiny RAG backend
- Planner / Retriever / Answer agents
- MCP-style tools and tool agent
- LangGraph workflow that routes between RAG and tools

This serves as a compact integrated example you can grow into a full system.
